In [48]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os
import acquire_prep as ap
import explore as ez
import modeling as mz

from env import get_connection

import matplotlib.pyplot as plt
import seaborn as sns
from pydataset import data
from math import sqrt
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoLars
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression
from sklearn.model_selection import train_test_split

# ACQUIRE

In [49]:
# create helper function to get the necessary connection url.
def get_db_connection(database):
    return get_connection(database)



In [50]:
# connect to sql zillow database

url = "zillow"

    # use this query to get data    
sql_query = '''SELECT *
            FROM predictions_2017
            JOIN properties_2017 on properties_2017.parcelid = predictions_2017.parcelid
            JOIN propertylandusetype on propertylandusetype.propertylandusetypeid = properties_2017.propertylandusetypeid;
            '''

    # assign data to data frame
df = pd.read_sql(sql_query, get_connection(url))

#filter out single family residential as requested
df= df[df['propertylandusedesc'] == 'Single Family Residential']


In [51]:
# acquiring, preparing, and splitting the zillow data
train, validate, test = ap.wrangle_zillow()

AttributeError: module 'acquire_prep' has no attribute 'wrangle_zillow'

## Acquire.Take away:

Data was acquired from CodeUp DB and 52442 observations and 6 features were used

# PREPARE 

In [16]:
df = acquire_db()

NameError: name 'get_connection' is not defined